In [9]:
import string
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def nettoyage(corpus_ensemble_documents):

    for i in range(len(corpus_ensemble_documents)):
        corpus_ensemble_documents[i] = corpus_ensemble_documents[i].lower()
    
    
    for i in range(len(corpus_ensemble_documents)):
        for c in string.punctuation:
            x = corpus_ensemble_documents[i].replace(c, ' ')
            corpus_ensemble_documents[i] = x
            
    
    stopwords_anglais = stopwords.words('english')
    for i in range(len(corpus_ensemble_documents)):
        L = corpus_ensemble_documents[i].split()
        
        L = [mot for mot in L if mot not in stopwords_anglais]
        corpus_ensemble_documents[i] = " ".join(L)
    return corpus_ensemble_documents

In [11]:
nettoyage(["This is a sample document.", "Another document, with punctuation!"])

['sample document', 'another document punctuation']

In [12]:
from numpy import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
def TF(terme, corpus, numero_document):
    x = corpus[numero_document].split().count(terme)
    y = len(corpus[numero_document].split())
    return x / y if y > 0 else 0

def IDF(terme, corpus, numero_document):
    """
    D : nombre total de documents dans le corpus
    d_t : nombre de documents dans lesquels le terme t apparaît
    This function calculates the Inverse Document Frequency (IDF) for a given term in a corpus of documents. 
    The IDF is a measure of how important a term is within the corpus.
    """
    D = len(corpus)
    d_t = 0
    for document in corpus:
        if terme in document:
            d_t += 1
    TF_val = TF(terme, corpus, numero_document)
    return TF_val * log(1+(D/d_t))

def cles_correspondante_a_valeur(valeur, dictionnaire):
    for cle in dictionnaire.keys():
        if dictionnaire[cle] == valeur:
            return cle

In [15]:
def matrice_sparce(dictionnaire, corpus_ensemble_documents):
    M = zeros((len(corpus_ensemble_documents), len(dictionnaire.values())))
    for i in range(len(corpus_ensemble_documents)):
        for j in dictionnaire.values():
            x = cles_correspondante_a_valeur(j, dictionnaire)
            M[i, j] = IDF(x, corpus_ensemble_documents, i)
    return M

def affiche(M):
    (n, p) = M.shape
    for i in range(n):
        for j in range(p):
            M[i, j] = round(M[i, j], 2)
    print(M)
    

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
text = ["La vie est douce", "La vie est tranquille, est belle, est douce", "le corona-virus est mechant"]

texte = nettoyage(text)
vect = TfidfVectorizer() 
T = vect.fit_transform(texte)
dictionnaire_des_mots = vect.vocabulary_
print("dictionnaire des mots : ", dictionnaire_des_mots)
liste_des_mots = list(dictionnaire_des_mots.keys())
print("liste des mots : ", liste_des_mots)
Matrice_sparce_correspondante = T.toarray()
print("Matrice_sparce_method predifine: \n")
affiche(Matrice_sparce_correspondante)
print("Matrice_sparce obtenue par notre methode: \n")
affiche(matrice_sparce(dictionnaire_des_mots, texte))

dictionnaire des mots :  {'la': 4, 'vie': 8, 'est': 3, 'douce': 2, 'tranquille': 7, 'belle': 0, 'le': 5, 'corona': 1, 'virus': 9, 'mechant': 6}
liste des mots :  ['la', 'vie', 'est', 'douce', 'tranquille', 'belle', 'le', 'corona', 'virus', 'mechant']
Matrice_sparce_method predifine: 

[[0.   0.   0.53 0.41 0.53 0.   0.   0.   0.53 0.  ]
 [0.38 0.   0.29 0.68 0.29 0.   0.   0.38 0.29 0.  ]
 [0.   0.48 0.   0.28 0.   0.48 0.48 0.   0.   0.48]]
Matrice_sparce obtenue par notre methode: 

[[0.   0.   0.23 0.17 0.23 0.   0.   0.   0.23 0.  ]
 [0.17 0.   0.11 0.26 0.11 0.   0.   0.17 0.11 0.  ]
 [0.   0.28 0.   0.14 0.   0.18 0.28 0.   0.   0.28]]


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [24]:
spams = pd.read_table("SMSSmapCollection.txt", sep='\t', header=None)
spams.columns = ['classe', 'message']

spams = spams.dropna(subset=['message'])

spamsTrain, spamsTest = train_test_split(spams, train_size=0.7, random_state=1)

parseur = TfidfVectorizer(binary=False)
XTrain = parseur.fit_transform(spamsTrain['message'])
XTest = parseur.transform(spamsTest['message'])

modelFirst = LogisticRegression()
modelFirst.fit(XTrain, spamsTrain['classe'])

score1 = modelFirst.score(XTrain, spamsTrain['classe'])
score2 = modelFirst.score(XTest, spamsTest['classe'])

print("Score train:", score1)
print("Score test:", score2)

Score train: 0.9853883619584722
Score test: 0.9784688995215312


In [25]:
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

parseurBis = CountVectorizer(stop_words='english')
XTrainBis = parseurBis.fit_transform(spamsTrain['message'])
mdtTrainBis = XTrainBis.toarray()
modelBis = LogisticRegression()
modelBis.fit(mdtTrainBis, spamsTrain['classe'])
mdtTestBis = parseurBis.transform(spamsTest['message'])
score3 = modelBis.score(mdtTestBis, spamsTest['classe'])
print("Score test avec CountVectorizer:", score3)
score4 = modelBis.score(mdtTrainBis, spamsTrain['classe'])
print("Score train avec CountVectorizer:", score4)


Score test avec CountVectorizer: 0.9844497607655502
Score train avec CountVectorizer: 0.9987182773647783


In [26]:
parseurTfidf = TfidfVectorizer()
XTrain3 = parseurTfidf.fit_transform(spamsTrain['message'])
mdtTrain3 = XTrain3.toarray()
model3 = LogisticRegression()
model3.fit(mdtTrain3, spamsTrain['classe'])
mdtTest3 = parseurTfidf.transform(spamsTest['message'])
score5 = model3.score(mdtTest3, spamsTest['classe'])
print("Score test avec TfidfVectorizer:", score5)
score6 = model3.score(mdtTrain3, spamsTrain['classe'])
print("Score train avec TfidfVectorizer:", score6)

Score test avec TfidfVectorizer: 0.9784688995215312
Score train avec TfidfVectorizer: 0.9853883619584722
